# 카메라 캘리브레이션(Camera Calibration) 개요
1. 카메라의 파라미터(Parameters)를 추정하는 과정
2. 카메라 내부 파라미터를 수식적인 변환 과정을 통해서 카메라의 왜곡을 보정하는 작업
3. 파라미터의 종류
* 내부 파라미터(Internal Parameters, 카메라/렌즈 시스템): 초점거리(Focal Length), 광학 중심(Optical Center), 렌즈의 방사 왜곡 계수(Radial Distortion Coefficients of the Lens)
* 외부 파라미터(External Parameters): 일부 세계 좌표계에 대한 카메라의 방향(회전 및 이동, Rotation and Translation)
4. 목표
* 알려진 3D 점과 해당 이미지 좌표 세트를 사용하여 3 * 3 행렬 k, 3* 3 회전 행렬 R, 3 * 1 이동 벡터 t를 찾는 것
* 내부와 외부 매개변수의 값을 얻을 때 카메라 캘리브레이션 진행 완료

# 이미지 형성의 기하학(Geometry of Image Formation)
1. 이미지 평면에 대한 3D 점의 투영을 찾기 위해 외부 파라미터(Rotation, Translation)를 사용하여 점을 **세계 좌표계** 에서 **카메라 좌표계** 로 변환
2. 카메라 픽셀 좌표에서 세계 좌표계의 3D 점으로 투영하는 과정
* 3D 점은 두 좌표계 간의 Rotation and Translation으로 구성된 **외부 행렬** 을 사용하여 세계 좌표에서 카메라 좌표로 변환
* 카메라 좌표계의 새로운 3D 점은 초점 거리, 광학 중심 등과 같은 내부 카메라 파라미터로 구성된 **내부 행렬** 을 사용하여 이미지 평면에 투영

# 카메라 캘리브레이션 진행 방법
1. 캘리브레이션 패턴(Calibration Pattern)
* 이미징 프로세스를 완전히 제어할 수 있는 경우, 캘리브레이션을 수행하는 가장 좋은 방법은 여러 관점에서 객체 또는 알려진 차원 패턴의 여러 이미지를 캡처하는 것
* 체커보드(Checkerboard) 기반 방식이 대표적.
* 체커보드 패턴 대신에 알려진 차원의 원형(Circular) 패턴 사용 가능
2. 기하학적 단서(Geometric Clues)
* 장면(Scene)에 종종 캘리브레이션에 사용할 수 있는 직선(Straight Lines) 및 소실점(Vanishing Point)과 같은 다른 기하학적 단서 존재
3. 딥 러닝 기반(Deep Learning based)
* 이미지 설정을 (거의) 제어할 수 없는 경우(예: 장면의 단일 이미지가 있는 경우), 딥러닝 기반 방법을 통해 카메라의 캘리브레이션 정보 획득 가능

# 카메라 캘리브레이션 진행 순서
1. 체커보드 패턴으로 실세계 좌표 정의
* 체커보드 패턴으로 이미지를 뚜렷하고 쉽게 감지 가능
* 체커보드 선의 교차점인 코너를 통해 위치 파악 용이
2. 다양한 시점에서 체커보드의 여러 이미지 캡쳐
3. 체커보드의 2D 좌표 찾기
* findChessboardCorners 내장 함수 사용
* cornerSubPix 사용하여 정밀하게 코너 위치 획득
4. 카메라 캘리브레이션
* 세계좌표의 3D 지점과 모든 이미지의 2D 위치를 OpenCV의 calibrate Camera 메서드로 전달

# 예제 코드

In [1]:
import numpy as np
import cv2
import glob

In [2]:
# 체커보드 차원 정의
# Defining the dimensions of checkerboard
CHECKERBOARD = (6, 9) # 체커보드 행과 열당 내부 코너 수
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

In [3]:
# 각 체커보드 이미지에 대한 3D 점 벡터를 저장할 벡터 생성
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []

In [4]:
# 각 체커보드 이미지에 대한 2D 점 벡터를 저장할 벡터 생성
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = []

In [5]:
# 3D 점의 세계 좌표 정의
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

In [6]:
# 주어진 디렉터리에 저장된 개별 이미지의 경로 추출
# Extracting path of individual image stored in a given directory
images = glob.glob('./Images/*.png')

for fname in images:
    img = cv2.imread(fname)
    # 그레이스케일로 변환
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # 체커보드 코너 찾기
    # Find the chess board corners
    # 이미지에서 원하는 개수의 코너가 발견되면 ret = true
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
    """
    원하는 개수의 코너가 감지되면,
    픽셀 좌표 미세 조정 및 체커보드도 이미지 표시
    """
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display
    them on the images of checker board
    """
    if ret == True:
        objpoints.append(objp)
        # 주어진 2D 점에 대한 픽셀 좌표 미세 조정
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)

        imgpoints.append(corners2)

        # 코너 그리기 및 표시
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)

    cv2.imshow('img', img)
    cv2.waitKey(0)

cv2.destroyAllWindows()

h,w = img.shape[:2] # 480, 640

In [7]:
# 알려진 3D 점(objpoints) 값과 감지된 코너의 해당 픽셀 좌표(imgpoints) 전달, 카메라 캘리브레이션 수행
"""
Performing camera calibration by
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

print("Camera matrix : \n") # 내부 카메라 행렬
print(mtx)
print("dist : \n") # 렌즈 왜곡 계수(Lens Distortion Coefficients)
print(dist)
print("rvecs : \n") # 회전 벡터
print(rvecs)
print("tvecs : \n") # 이동 벡터
print(tvecs)

Camera matrix : 

[[2.59816083e+17 0.00000000e+00 9.14500000e+02]
 [0.00000000e+00 2.59883058e+17 6.64500000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[-6.03924728e-17 -4.60847360e-33 -1.39825059e-08  2.37092202e-08
  -3.90376842e-49]]
rvecs : 

(array([[ 1.577021  ],
       [-0.30956547],
       [-0.58118676]]),)
tvecs : 

(array([[-2.75128992e+00],
       [ 1.74775986e+00],
       [ 4.05243759e+08]]),)
